In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
import numpy as np

In [3]:
import time

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [5]:
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.wordnet import WordNetLemmatizer

In [6]:
# porter_stemmer = PorterStemmer()

In [7]:
# wordnet_lemmatizer = WordNetLemmatizer()

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
stop_words.update(['read', 'more', 'subscribe', 'twitter', 'google', 'facebook', 'home', 'advertise', 'rss', 'none', 'privacy', 'about', 'us', 'contact', 'login', 'signup', 'sign', 'up', 'register', 'search', 'terms', 'condition', 'faqs', 'disclaimer', 'account'])

In [10]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [11]:
train_df['Content'] = np.nan
test_df['Content'] = np.nan

In [12]:
train_df.head()

,Webpage_id,Domain,Url,Tag,Content
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news,NaN
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news,NaN
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news,NaN
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news,NaN
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news,NaN


In [13]:
import sys
import csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

In [14]:
maxInt

922337203

In [15]:
'''
i = 0
for chunk in pd.read_csv('../input/html_data.csv', chunksize=100, encoding='utf8', engine='python'):
    for i, row in chunk.iterrows():
        i = i + 1
'''

"\ni = 0\nfor chunk in pd.read_csv('../input/html_data.csv', chunksize=100, encoding='utf8', engine='python'):\n    for i, row in chunk.iterrows():\n        i = i + 1\n"

In [16]:
start = time.time()
i = 0
for chunk in pd.read_csv('../input/html_data.csv', chunksize=5000, encoding='utf8', engine='python'):
    # print(chunk['Webpage_id'][0])
    # print('##################################')
    for j, row in chunk.iterrows():    
        soup = BeautifulSoup(row['Html'])

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        xyz = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = ' '.join(abc for abc in xyz if abc)

        word_tokens = word_tokenize(text.lower())

        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                if w.isalpha() and len(w)!=1:
                    # w = porter_stemmer.stem(w)
                    # w = wordnet_lemmatizer.lemmatize(w)
                    filtered_sentence.append(w)
        # print(len(filtered_sentence))
        # print(' '.join(filtered_sentence))
        train_df.loc[(train_df['Webpage_id'] == row['Webpage_id']), 'Content'] = ' '.join(filtered_sentence)
        test_df.loc[(test_df['Webpage_id'] == row['Webpage_id']), 'Content'] = ' '.join(filtered_sentence)

        '''
        for link in soup.findAll('a'):
            print(link.string)
        '''
        i = i + 1
        # print(chunk['Html'][0])
end = time.time()
print(end-start)
print(i)

6906.396077871323
79345


In [17]:
6906/3600

1.9183333333333332

In [18]:
train_df.shape

(53447, 5)

In [19]:
train_df.isnull().sum()

Webpage_id    0
Domain        0
Url           0
Tag           0
Content       0
dtype: int64

In [21]:
train_df.head()

,Webpage_id,Domain,Url,Tag,Content
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news,tecfidera gilenya aubagio battle ms share heat...
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news,novo equipped weather storm diabetes market ce...
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news,another exec departs troubled endo time anothe...
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news,would teva buy korea celltrion beef biosimilar...
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news,allergan recruits actress marisa tomei drive d...


In [22]:
test_df.shape

(25787, 4)

In [23]:
test_df.head()

,Webpage_id,Domain,Url,Content
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413,isrctn development validation caregiver qualit...
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,clinical trials eu clinical trials help joinin...
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,clinical trials eu clinical trials help joinin...
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,clinical trials eu clinical trials help joinin...
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,clinical trials eu clinical trials help joinin...


In [26]:
train_df.to_csv('../input/train_cont.csv', index=False, encoding='utf8')

In [27]:
test_df.to_csv('../input/test_cont.csv', index=False, encoding='utf8')